In [ ]:
import requests
import pandas as pd
import geopandas as gpd
from bs4 import BeautifulSoup
from shapely.ops import transform
from shapely.geometry import MultiPolygon, MultiLineString, MultiPoint, Polygon, LineString, Point
from io import BytesIO
from tqdm import tqdm
import time
from random import uniform
import zipfile

### Important functions

In [ ]:
def kmz_to_kml(content):
    try:
        with zipfile.ZipFile(BytesIO(content)) as kmz:
            with kmz.open(next(f for f in kmz.namelist() if f.endswith('.kml'))) as kml_file:
                return kml_file.read().decode('utf-8')
    except zipfile.BadZipFile:
        return content.decode('utf-8')

# Function to fetch and parse KML file
def fetch_kml(uri):
    response = requests.get(uri)
    if response.status_code == 200:
        return response.content
    else:
        print(f"Failed to download KML file from {uri}")
        return None

# Function to parse KML and convert to geometries
def parse_kml(content):
    soup = BeautifulSoup(content, 'xml')
    geometries = []

    # Find all MultiGeometry elements, which can host multiple Polygons
    for multi_geom in soup.find_all('MultiGeometry'):
        polygons = []
        linestrings = []
        points = []
        for polygon in multi_geom.find_all('Polygon'):
            if polygon.find('coordinates').string == None:
                polygons.append(Polygon())
                continue
            coords = polygon.find('coordinates').string.strip().split()
            c_points = [tuple(map(float, c.split(','))) for c in coords]
            polygons.append(Polygon(c_points))
        for linestring in multi_geom.find_all('LineString'):
            if linestring.find('coordinates').string == None:
                linestrings.append(LineString())
                continue
            coords = linestring.find('coordinates').string.strip().split()
            c_points = [tuple(map(float, c.split(','))) for c in coords]
            if len(c_points) != 1:
                linestrings.append(LineString(c_points))
            else:
                geometries.append(Point(c_points))
        for point in multi_geom.find_all('Point'):
            if point.find('coordinates').string == None:
                points.append(Point())
                continue
            coords = point.find('coordinates').string.strip().split()
            c_points = [tuple(map(float, c.split(','))) for c in coords]
            points.append(Point(c_points))
        if polygons:
            geometries.append(MultiPolygon(polygons))
        if linestrings:
            geometries.append(MultiLineString(linestrings))
        if points:
            geometries.append(MultiPoint(points))

    # Also check for geometries that are not part of MultiGeometry
    # Check for Polygons
    for polygon in soup.find_all('Polygon'):
        if polygon.parent.name != 'MultiGeometry':
            if polygon.find('coordinates').string == None:
                geometries.append(Polygon())
                continue
            coords = polygon.find('coordinates').string.strip().split()
            c_points = [tuple(map(float, c.split(','))) for c in coords]
            geometries.append(Polygon(c_points))

    # Check for LineStrings
    for linestring in soup.find_all('LineString'):
        if linestring.parent.name != 'MultiGeometry':
            if linestring.find('coordinates').string == None:
                geometries.append(LineString())
                continue
            coords = linestring.find('coordinates').string.strip().split()
            c_points = [tuple(map(float, c.split(','))) for c in coords]
            if len(c_points) != 1:
                geometries.append(LineString(c_points))
            else:
                geometries.append(Point(c_points))

    # Check for Points
    for point in soup.find_all('Point'):
        if point.parent.name != 'MultiGeometry':
            if point.find('coordinates').string == None:
                geometries.append(Point())
                continue
            coords = point.find('coordinates').string.strip().split()
            c_points = [tuple(map(float, c.split(','))) for c in coords]
            geometries.append(Point(c_points))

    return geometries

# Main processing
def process_kml_uris(kml_uris):
    all_geometries = []
    for uri in kml_uris:
        uri_content = fetch_kml(uri)
        if uri_content:
            kml_content = kmz_to_kml(uri_content)
            geometries = parse_kml(kml_content)
            all_geometries.extend(geometries)
    return all_geometries

### Read project list

Project lists were acquired from the Verra registry at July 8th and 9th, 2024 by using the export to csv functionality for bulk download:

- Verified Carbon Standard (VCS): https://registry.verra.org/app/search/VCS/All%20Projects
- Climate, Community & Biodiversity Standards (CCB): https://registry.verra.org/app/search/CCB/All%20Projects
- Sustainable Development Verified Impact Standard (VISta): https://registry.verra.org/app/search/SDVISTA/All%20Projects

In [ ]:
projects_vcs = pd.read_csv('/Users/tillkoebe/Documents/GitHub/Forest_Monitoring/input/Verra/allprojects_vcs.csv')
projects_ccb = pd.read_csv('/Users/tillkoebe/Documents/GitHub/Forest_Monitoring/input/Verra/allprojects_ccb.csv')
projects_vista = pd.read_csv('/Users/tillkoebe/Documents/GitHub/Forest_Monitoring/input/Verra/allprojects_vista.csv')

In [ ]:
print(projects_vcs.shape, projects_ccb.shape, projects_vista.shape)

In [ ]:
projects_vcs.dropna(subset = 'AFOLU Activities', inplace = True)
projects_ccb.dropna(subset = 'CCB Project Type', inplace = True)
projects_vista.dropna(subset = 'Project Type', inplace = True)

In [ ]:
print(projects_vcs.shape, projects_ccb.shape, projects_vista.shape)

In [ ]:
project_list_vcs = projects_vcs[projects_vcs['AFOLU Activities'].str.contains("ARR")].ID.tolist()
project_list_ccb = projects_ccb[projects_ccb['CCB Project Type'].str.contains("Afforestation, Reforestation and Revegetation")].ID.tolist()
project_list_vista = projects_vista[projects_vista['Project Type'].str.contains("Agriculture Forestry and Other Land Use")].ID.tolist()

In [ ]:
project_list = list(set(project_list_vcs + project_list_ccb + project_list_vista))

In [ ]:
print(len(project_list), len(project_list_vcs), len(project_list_ccb), len(project_list_vista))

### Extract geometries per project

In [ ]:
project_gdf = pd.DataFrame()
no_geom_list = []

In [ ]:
for project_id in tqdm(project_list):

    try:
        response = requests.get(f'https://registry.verra.org/uiapi/resource/resourceSummary/{project_id}')

    except Exception as e:
        print(f"Error with project {project_id}: {e}")
        continue

    if response.status_code == 200:
        data = response.json()
        
        # Extract KML URIs
        kml_uris = []
        for group in data.get('documentGroups', []):
            for document in group.get('documents', []):
                if document['documentType'].lower() == 'kml file' or document['documentName'].endswith('.kml'):
                    kml_uris.append(document['uri'])
        if kml_uris:
            kml_uris = list(set(kml_uris))
            try:
                # Process the KML URIs to get geometries
                geometries = process_kml_uris(kml_uris)
    
            except Exception as e:
                print(f"Error querying the geometry of project {project_id}: {e}")
                continue
        else:
            no_geom_list.append(project_id)
            print(f'No geometries available for project: {project_id}')

        # Convert geometries to GeoPandas DataFrame
        temp = gpd.GeoDataFrame(geometry=geometries)
        
        # Assign CRS
        if abs(temp.geometry.centroid.y).max() > 180:
            temp = temp.set_crs(3857).to_crs(4326)
        else:
            temp = temp.set_crs(4326)

        # Explode MultiPolygons into individual Polygons
        temp = temp.explode(index_parts=False)
        
        # 3D to 2D geometries
        temp['geometry'] = temp['geometry'].apply(lambda geometry: transform(lambda x, y, z=None: (x, y), geometry))
        
        # Assign identifiers
        temp['project_id'] = project_id
        if data['description']:
            temp['project_description'] = data['description']
        else:
            temp['project_description'] = None
        temp = temp.reset_index(drop = True).reset_index().rename(columns={'index': 'site_id'})
        
        # Add project to output
        project_gdf = pd.concat([project_gdf, temp], ignore_index=True)
        
        # Delay to avoid excess request responses
        time.sleep(uniform(0, 2.0))
        
    else:
        print(f"Request failed with status code: {response.status_code}")


Check which project ids are not included

In [ ]:
set(project_list) - set(project_gdf['project_id']) - set(no_geom_list)

In [ ]:
project_list = set(project_list) - set(project_gdf['project_id']) - set(no_geom_list)

!! Important: Re-run function above to ensure all projects have been queried !!

In [ ]:
project_gdf.project_id.nunique()

In [ ]:
project_gdf.info()

In [ ]:
project_gdf['geometry'] = project_gdf['geometry'].make_valid()

In [ ]:
project_gdf = project_gdf.explode(index_parts=False).explode(index_parts=False).reset_index(drop = True)

In [ ]:
project_gdf['geometry'] = project_gdf['geometry'].apply(
    lambda geom: Polygon(list(geom.coords) + [geom.coords[0]]) if isinstance(geom, LineString) and not geom.is_closed and len(geom.coords) > 0 else
                 Polygon(geom.coords) if isinstance(geom, LineString) and geom.is_closed else
                 geom
)

In [ ]:
project_gdf['geometry'] = project_gdf['geometry'].make_valid()

In [ ]:
project_gdf['sites_sqkm'] = project_gdf.to_crs(3857).area/1e6

In [ ]:
project_gdf.sites_sqkm.describe()

Add project-level metadata

In [ ]:
projects_df = (pd.concat([projects_vcs[['ID', 'Name', 'Status', 'Country/Area']], 
                         projects_ccb[['ID', 'Name', 'Status', 'Country/Area']], 
                         projects_vista[['ID', 'Name', 'Status', 'Country/Area']]])
               .drop_duplicates(subset = 'ID')
               .rename(columns = {'ID':'project_id', 'Name':'project_name', 'Status':'status_reported', 'Country/Area':'country_reported'}))

In [ ]:
project_gdf = project_gdf.merge(projects_df, on = 'project_id', how = 'left')

In [ ]:
project_gdf.info()

In [ ]:
project_gdf.to_file("/Users/tillkoebe/Documents/GitHub/Forest_Monitoring/input/Verra/verra_sites.gpkg", driver="GPKG")